## Data Preparation

TODO: add text about data set (summary from data exploration)

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os

#include utils directory
import sys
sys.path.append('..')

from utils.files import *
DATA_PATH = os.path.join('..', 'data')


#### Teams & Coaches

In [8]:
### Teams

teams_df = pd.read_csv(os.path.join(DATA_PATH, DATA_TEAMS))
merged_df = teams_df[['year', 'tmID', 'playoff']]

teams_df = prepareTeamsDf(teams_df)

# Merge
teams_df['year'] = teams_df['year'] + 1
teams_df['playoff_prev'] = teams_df['playoff']
teams_df['rank_prev'] = teams_df['rank']
teams_df.drop(columns=['playoff', 'rank'], inplace=True)
merged_df = pd.merge(merged_df, teams_df, on=['year', 'tmID'], how='left')

display(merged_df)

### Coaches
coaches_df = pd.read_csv(os.path.join(DATA_PATH, DATA_COACHES))
# Collapse wins & losses into one feature
coaches_df['coachWLRatio'] = (coaches_df['won'] + coaches_df['post_wins']) / (coaches_df['lost'] + coaches_df['post_losses'])
# Remove unnecessary info
coaches_df.drop(columns=['lgID', 'stint', 'won', 'lost', 'post_wins', 'post_losses'], inplace=True)
# Merge
coaches_temp = coaches_df[['coachID', 'year', 'tmID']]
merged_df = pd.merge(merged_df, coaches_temp, on=['year', 'tmID'], how='left')
coaches_df['year'] = coaches_df['year'] + 1
coaches_df.drop(columns=['tmID'], inplace=True)
merged_df = pd.merge(merged_df, coaches_df, on=['year', 'coachID'], how='left')
merged_df['coachWLRatio'] = merged_df['coachWLRatio'].fillna(merged_df['coachWLRatio'].mean())

,year,tmID,playoff,teamWLRatio,fgm,fga,ftm,fta,3pm,3pa,reb,asts,pf,stl,to,blk,pts,playoff_prev,rank_prev
0,9,ATL,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,ATL,Y,0.133333,1909.0,4512.0,1221.0,1643.0,374.0,1100.0,2342.0,1176.0,1522.0,595.0,1154.0,276.0,5413.0,N,7.0
2,1,CHA,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,CHA,Y,0.333333,1691.0,3833.0,964.0,1293.0,269.0,809.0,1925.0,1147.0,1309.0,481.0,922.0,213.0,4615.0,N,8.0
4,3,CHA,Y,1.285714,1478.0,3626.0,841.0,1090.0,267.0,797.0,1859.0,910.0,1184.0,474.0,921.0,238.0,4064.0,Y,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,6,WAS,N,1.000000,1737.0,4101.0,984.0,1347.0,251.0,708.0,2169.0,1001.0,1279.0,492.0,916.0,239.0,4709.0,Y,4.0
138,7,WAS,Y,0.888889,1680.0,3840.0,877.0,1194.0,331.0,943.0,1967.0,1062.0,1335.0,465.0,939.0,184.0,4568.0,N,5.0
139,8,WAS,N,1.125000,1925.0,4301.0,1163.0,1566.0,391.0,1075.0,2141.0,1159.0,1471.0,558.0,1114.0,202.0,5404.0,Y,4.0
140,9,WAS,N,0.888889,1801.0,4386.0,1284.0,1640.0,338.0,1048.0,2326.0,1021.0,1525.0,538.0,1150.0,236.0,5224.0,N,5.0


#### Players and Awards

In [9]:
# Players
players_teams_df = pd.read_csv(os.path.join(DATA_PATH, DATA_PLAYERS_TEAMS))

merging = ['minutes', 'points', 'oRebounds', 'dRebounds', 'assists', 'steals', 'blocks', 'turnovers',
           'PF', 'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade', 'dq']
new_pt_df = pd.DataFrame()

# Join the post and regular season stats
hasUpper = lambda s: any(x.isupper() for x in s)
for col in merging:
    postName = 'Post' + (col if hasUpper(col) else col.capitalize())
    if col == 'dq':
        postName = 'PostDQ'
    new_pt_df[col] = players_teams_df[col] + players_teams_df[postName]

# Make everything a ratio of minutes
for col in new_pt_df:
    if col == 'minutes':
        continue
    new_pt_df[col] = new_pt_df[col] / new_pt_df['minutes']
new_pt_df.drop(columns=['minutes'], inplace=True)

for col in ['playerID', 'year', 'tmID']:
    new_pt_df[col] = players_teams_df[col]

display(new_pt_df)

# get the average stats for players the previous year
for i in range(len(merged_df)):
    cur = new_pt_df[(new_pt_df['tmID'] == merged_df['tmID'][i])
              & (new_pt_df['year'] == merged_df['year'][i] - 1)].copy()
    cur.drop(columns=['playerID', 'year', 'tmID'], inplace=True)
    for col in cur:
        merged_df[col] = cur[col].sum()
        #pass

'''
awards_df = pd.read_csv(os.path.join(DATA_PATH, DATA_AWARDS))
awards_df.drop(columns=['lgID'], inplace=True)

players_df = pd.read_csv(os.path.join(DATA_PATH, DATA_PLAYERS))
players_df.rename(columns={"bioID": "playerID"}, inplace=True)
# merged_df = pd.merge(merged_df, players_df, on=['playerID'], how='inner')
#merged_df = pd.merge(merged_df, players_teams_df, on=['tmID','year'], how='left')

# merged_df = pd.merge(merged_df, awards_df, on=['playerID', 'year'], how='left')
'''

,points,oRebounds,dRebounds,assists,steals,blocks,turnovers,PF,fgAttempted,fgMade,ftAttempted,ftMade,threeAttempted,threeMade,dq,playerID,year,tmID
0,0.405437,0.050827,0.154846,0.062648,0.049645,0.010638,0.100473,0.082742,0.346336,0.134752,0.156028,0.113475,0.089835,0.022459,0.002364,abrossv01w,2,MIN
1,0.390062,0.055901,0.125466,0.074534,0.052174,0.012422,0.114286,0.090683,0.392547,0.147826,0.144099,0.069565,0.074534,0.024845,0.000000,abrossv01w,3,MIN
2,0.396051,0.052265,0.117305,0.099884,0.055749,0.013937,0.113821,0.101045,0.356562,0.137050,0.123113,0.089431,0.103368,0.032520,0.000000,abrossv01w,4,MIN
3,0.313800,0.037807,0.119093,0.090737,0.058601,0.007561,0.086957,0.092628,0.306238,0.107750,0.094518,0.056711,0.115312,0.041588,0.000000,abrossv01w,5,MIN
4,0.391248,0.037323,0.100386,0.077220,0.061776,0.007722,0.102960,0.110682,0.355212,0.140283,0.093951,0.068211,0.105534,0.042471,0.000000,abrossv01w,6,MIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,0.162162,0.000000,0.081081,0.000000,0.000000,0.000000,0.108108,0.108108,0.162162,0.054054,0.108108,0.054054,0.000000,0.000000,0.000000,zakalok01w,3,PHO
1872,0.230068,0.025057,0.063781,0.120729,0.036446,0.002278,0.104784,0.066059,0.207289,0.086560,0.052392,0.043280,0.052392,0.013667,0.000000,zarafr01w,6,SEA
1873,0.511088,0.034847,0.103485,0.080253,0.034847,0.021119,0.088701,0.133052,0.385428,0.149947,0.219641,0.187962,0.069694,0.023231,0.003168,zellosh01w,10,DET
1874,0.366667,0.000000,0.066667,0.033333,0.000000,0.000000,0.066667,0.100000,0.133333,0.066667,0.200000,0.200000,0.066667,0.033333,0.000000,zirkozu01w,4,WAS


'\nawards_df = pd.read_csv(os.path.join(DATA_PATH, DATA_AWARDS))\nawards_df.drop(columns=[\'lgID\'], inplace=True)\n\nplayers_df = pd.read_csv(os.path.join(DATA_PATH, DATA_PLAYERS))\nplayers_df.rename(columns={"bioID": "playerID"}, inplace=True)\n# merged_df = pd.merge(merged_df, players_df, on=[\'playerID\'], how=\'inner\')\n#merged_df = pd.merge(merged_df, players_teams_df, on=[\'tmID\',\'year\'], how=\'left\')\n\n# merged_df = pd.merge(merged_df, awards_df, on=[\'playerID\', \'year\'], how=\'left\')\n'

#### Playoffs

In [10]:


'''
# Merge Series_post and Teams twice (for winning and losing teams)
# using "tmIDWinner" and "lgIDWinner" keys for winning teams
winning_teams_df = pd.merge(series_post_df, teams_df, left_on=['year', 'tmIDWinner', 'lgIDWinner'], right_on=[
                            'year', 'tmID', 'lgID'], suffixes=('', '_winning'), how='inner')
# using "tmIDLoser" and "lgIDLoser" keys for losing teams
losing_teams_df = pd.merge(series_post_df, teams_df, left_on=['year', 'tmIDLoser', 'lgIDLoser'], right_on=[
                           'year', 'tmID', 'lgID'], suffixes=('', '_losing'), how='inner')

'''
# Merge Teams_post and Teams using the combination of "year," "tmID," and "lgID" keys
# merged_df = pd.merge(merged_df, teams_post_df, on=['year', 'tmID', 'lgID'], how='inner')





'\n# Merge Series_post and Teams twice (for winning and losing teams)\n# using "tmIDWinner" and "lgIDWinner" keys for winning teams\nwinning_teams_df = pd.merge(series_post_df, teams_df, left_on=[\'year\', \'tmIDWinner\', \'lgIDWinner\'], right_on=[\n                            \'year\', \'tmID\', \'lgID\'], suffixes=(\'\', \'_winning\'), how=\'inner\')\n# using "tmIDLoser" and "lgIDLoser" keys for losing teams\nlosing_teams_df = pd.merge(series_post_df, teams_df, left_on=[\'year\', \'tmIDLoser\', \'lgIDLoser\'], right_on=[\n                           \'year\', \'tmID\', \'lgID\'], suffixes=(\'\', \'_losing\'), how=\'inner\')\n\n'

In [11]:
merged_df['playoff'] = merged_df['playoff'].eq('Y').mul(1)
merged_df = merged_df.select_dtypes(['number']) # Remove later
merged_df.dropna(axis=0, inplace=True)
merged_df.head()

print(merged_df.shape)
merged_df.head()

(140, 33)


,year,playoff,teamWLRatio,fgm,fga,ftm,fta,3pm,3pa,reb,...,blocks,turnovers,PF,fgAttempted,fgMade,ftAttempted,ftMade,threeAttempted,threeMade,dq
1,10,1,0.133333,1909.0,4512.0,1221.0,1643.0,374.0,1100.0,2342.0,...,0.173121,1.541965,1.708185,4.769913,1.759365,1.201718,0.754538,1.360963,0.431455,0.009553
3,2,1,0.333333,1691.0,3833.0,964.0,1293.0,269.0,809.0,1925.0,...,0.173121,1.541965,1.708185,4.769913,1.759365,1.201718,0.754538,1.360963,0.431455,0.009553
4,3,1,1.285714,1478.0,3626.0,841.0,1090.0,267.0,797.0,1859.0,...,0.173121,1.541965,1.708185,4.769913,1.759365,1.201718,0.754538,1.360963,0.431455,0.009553
5,4,1,1.285714,1548.0,3597.0,934.0,1261.0,344.0,899.0,1870.0,...,0.173121,1.541965,1.708185,4.769913,1.759365,1.201718,0.754538,1.360963,0.431455,0.009553
6,5,0,1.125000,1577.0,3783.0,933.0,1257.0,325.0,924.0,1978.0,...,0.173121,1.541965,1.708185,4.769913,1.759365,1.201718,0.754538,1.360963,0.431455,0.009553


In [12]:
# Save the result to a new CSV file
merged_df.to_csv(os.path.join(DATA_PATH, DATA_MERGED), index=False)